In [15]:
# importação de dados
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
#tratando os outliers

def tratar_outliers(df, colunas = None):
  """
  trata outliers para colunas numéricas

  parâmetros:
  df: dataframe a ser inspecionado
  colunas: lista de colunas ou None (aplica em todas as numéricas)

  """
  if colunas is None:
    colunas = df.select_dtypes(include = ['number']).columns

  for col in colunas:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr

    df[col] = np.where(
        (df[col] < limite_inferior) | (df[col] > limite_superior),
        np.nan,
        df[col]
    )
  return df

#extraindo o mes do dataset

def extrai_mes(df):
  df = df.copy()
  data = df["Data"]
  data = pd.to_datetime(data)
  df["mes"] = data.dt.month
  return df

#gerando as metricas mensais
def gerar_metricas_mensais(df, metricas):
    """
    Nova função para agregação em duas etapas (diária + mensal)
    """
    try:
        df_temp = df.copy()
        df_temp['datetime'] = pd.to_datetime(df_temp['datetime'])
        df_temp = df_temp.set_index('datetime')

        # Agregação diária
        daily_agg = {col: ops['diario'] for col, ops in metricas.items()}
        df_diario = df_temp.resample('D').agg(daily_agg)

        # Agregação mensal
        monthly_agg = {col: ops['mensal'] for col, ops in metricas.items()}
        df_mensal = df_diario.resample('M').agg(monthly_agg)

        # Renomear colunas
        new_columns = [f"{col}_{metricas[col]['diario']}_{metricas[col]['mensal']}"
                      for col in df_mensal.columns]
        df_mensal.columns = new_columns

        return df_mensal.reset_index()

    except Exception as e:
        print(f"Erro na agregação mensal: {str(e)}")
        return None

def ler_arquivo(nome_arquivo):
    try:
        # Tenta com UTF-8 (codificação padrão)
        df = pd.read_csv(nome_arquivo, sep=';', skiprows=8, decimal=',', encoding='utf-8')
    except UnicodeDecodeError:
        # Se falhar, tenta Latin-1 (compatível com outros casos)
        df = pd.read_csv(nome_arquivo, sep=';', skiprows=8, decimal=',', encoding='latin-1')
    return df

In [17]:
#extraindo o dataframe

FILEPATH = "/content/INMET_N_AC_A104_RIO_BRANCO_01-01-2023_A_31-12-2023.CSV"

df = ler_arquivo(FILEPATH)
display(df.head(10))

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2023/01/01,0000 UTC,0.0,990.8,990.8,989.9,0.0,24.2,21.9,24.9,24.0,21.9,21.0,88.0,79.0,87.0,12.0,3.9,0.3,NaN
1,2023/01/01,0100 UTC,0.0,992.2,992.2,990.8,0.0,23.3,21.8,24.2,23.3,22.2,21.8,91.0,87.0,91.0,54.0,1.5,0.0,NaN
2,2023/01/01,0200 UTC,0.0,993.1,993.2,992.2,0.0,23.4,22.1,23.6,23.2,22.1,21.8,92.0,91.0,92.0,13.0,0.7,0.0,NaN
3,2023/01/01,0300 UTC,0.0,993.6,993.6,993.1,0.0,23.1,21.9,23.5,23.0,22.1,21.8,93.0,92.0,93.0,28.0,0.0,0.0,NaN
4,2023/01/01,0400 UTC,0.0,993.0,993.7,993.0,0.0,23.3,21.9,23.5,23.1,22.3,21.9,93.0,92.0,92.0,27.0,0.0,0.0,NaN
5,2023/01/01,0500 UTC,0.0,992.7,993.0,992.7,0.0,23.0,22.0,23.3,22.9,22.0,21.8,94.0,92.0,94.0,14.0,0.0,0.0,NaN
6,2023/01/01,0600 UTC,0.0,992.2,992.7,992.2,0.0,22.9,21.9,23.3,22.9,22.3,21.8,94.0,93.0,94.0,31.0,0.0,0.0,NaN
7,2023/01/01,0700 UTC,0.0,991.7,992.2,991.7,0.0,23.0,21.9,23.1,22.9,22.1,21.9,94.0,93.0,93.0,19.0,0.0,0.0,NaN
8,2023/01/01,0800 UTC,0.0,991.3,991.7,991.3,0.0,23.1,21.9,23.1,22.9,22.0,21.8,94.0,93.0,93.0,13.0,1.5,0.0,NaN
9,2023/01/01,0900 UTC,0.0,991.4,991.5,991.3,0.0,23.1,22.0,23.1,23.0,22.0,21.8,94.0,93.0,94.0,23.0,1.6,0.0,NaN


In [18]:
#mostrando as colunas

display(df.columns)

Index(['Data', 'Hora UTC', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
       'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
       'RADIACAO GLOBAL (Kj/m²)',
       'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
       'TEMPERATURA DO PONTO DE ORVALHO (°C)',
       'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
       'UMIDADE RELATIVA DO AR, HORARIA (%)',
       'VENTO, DIREÇÃO HORARIA (gr) (° (gr))', 'VENTO, RAJADA MAXIMA (m/s)',
       'VENTO, VELOCIDADE HORARIA (m/s)', 'Unnamed: 19'],
      dtype='object')

In [19]:
#pegando o mes
df_mes = extrai_mes(df)
display(df_mes.tail(10))

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),...,TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19,mes
8750,2023/12/31,1400 UTC,0.0,995.8,995.8,995.6,764.4,26.9,23.6,26.9,...,23.8,23.1,86.0,82.0,82.0,23.0,4.0,0.9,NaN,12
8751,2023/12/31,1500 UTC,0.0,995.8,995.9,995.7,1553.4,28.6,24.1,28.6,...,24.2,23.3,83.0,76.0,77.0,2.0,2.9,0.9,NaN,12
8752,2023/12/31,1600 UTC,0.0,995.3,995.8,995.3,2430.4,29.6,23.8,29.6,...,24.3,23.2,80.0,69.0,71.0,7.0,4.7,1.3,NaN,12
8753,2023/12/31,1700 UTC,0.0,994.1,995.3,994.1,3548.0,30.5,23.0,31.6,...,24.4,22.5,72.0,61.0,64.0,22.0,4.0,1.7,NaN,12
8754,2023/12/31,1800 UTC,0.0,992.5,994.1,992.5,2278.4,31.6,23.6,32.3,...,24.7,22.8,67.0,59.0,62.0,341.0,4.0,0.8,NaN,12
8755,2023/12/31,1900 UTC,0.0,991.2,992.5,991.1,1936.1,30.5,22.9,32.3,...,25.1,22.9,67.0,59.0,64.0,344.0,7.2,1.5,NaN,12
8756,2023/12/31,2000 UTC,13.6,990.4,991.3,990.3,315.8,25.5,24.1,30.5,...,25.1,23.2,92.0,64.0,92.0,44.0,7.2,0.3,NaN,12
8757,2023/12/31,2100 UTC,0.2,990.5,991.0,990.4,132.8,25.5,24.1,25.6,...,24.2,23.8,92.0,91.0,92.0,356.0,4.5,0.3,NaN,12
8758,2023/12/31,2200 UTC,0.0,990.6,990.9,990.5,158.9,25.1,23.7,25.7,...,24.3,23.5,92.0,90.0,92.0,92.0,2.1,0.4,NaN,12
8759,2023/12/31,2300 UTC,0.0,992.0,992.0,990.6,54.8,24.7,23.4,25.4,...,23.9,23.4,93.0,90.0,93.0,38.0,2.6,0.1,NaN,12


In [20]:
dict_cols = {
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "sum",
    "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)":"mean",
    "RADIACAO GLOBAL (Kj/m²)":"mean",
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)":"mean",
    "TEMPERATURA DO PONTO DE ORVALHO (°C)":"mean",
    "UMIDADE RELATIVA DO AR, HORARIA (%)":"mean",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)":["mean", "max", "min"],
    "VENTO, VELOCIDADE HORARIA (m/s)":["mean","max"],
    "mes":"count"
}

AGREGADOR = "mes"
df_agg = df.groupby(AGREGADOR).agg(
  cli_precip_dia_med = ("mes","count"),
  cli_precip_total = ("PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","sum"),
  cli_pressao_atm_med = ("PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)","mean"),
  cli_rad_global_med = ("RADIACAO GLOBAL (Kj/m²)","mean"),
  cli_temp_ar_med = ("TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)","mean"),
  cli_temp_orvalho_med = ("TEMPERATURA DO PONTO DE ORVALHO (°C)","mean"),
  cli_umid_rel_ar_med = ("UMIDADE RELATIVA DO AR, HORARIA (%)","mean"),
  cli_umid_rel_min_med = ("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)","mean"),
  cli_umid_rel_min_max = ("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)","max"),
  cli_umid_rel_min_min = ("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)","min"),
  cli_vento_veloc_max = ("VENTO, VELOCIDADE HORARIA (m/s)","max"),
  cli_vento_veloc_med = ("VENTO, VELOCIDADE HORARIA (m/s)","mean")


)

df_agg["cli_precip_dia_med"] = df_agg["cli_precip_total"]*24/df_agg["cli_precip_dia_med"]
display(df_agg.head(15))

KeyError: 'mes'

In [ ]:
#colunas do novo dataframe (que mostra um mes)
display(df_agg.columns)


In [ ]:
dict_nomes = {
    "cli_precip_dia_med": "Precipitação Média",
    "cli_precip_total": "Precipitação Total",
    "cli_pressao_atm_med": "Pressão Média",
    "cli_rad_global_med": "Radiação Média",
    "cli_temp_ar_med": "Temperatura Média",
    "cli_temp_orvalho_med": "Temperatura Orvalho Média",
    "cli_umid_rel_ar_med": "Umidade Média",

}